Zadanie opiera się na danych dostępnych ogólnie pod linkiem:
http://stat-computing.org/dataexpo/2009/the-data.html
1. Stwórz set z latami 2007 i 2008
2. Scharakteryzuj wybranego przewoźnika (UniqueCarrier) na przestrzeni tych dwóch lat na podstawie dostępnych danych.
3. Wiążąc kilka zmiennych ze sobą, wykaż ciekawą (we własnej ocenie) zależność dla jednego carriera lub dla całej grupy

## Task #1 : Stwórz set z latami 2007 i 2008
We want to import two datasets, for year 2007 and 2008. Data is located <a href="http://stat-computing.org/dataexpo/2009/the-data.html">here</a>. Data is compressed using bzip copression format. We want to avoid downloading the data to our hard-drive (cause it would hinder this notebook's code reusability), therefore we'll attempt to download data, decompress it and assign it to Pandas.DataFrame using built-in Pandas method - read_csv(URL, compression=""bz2").


In [1]:
import sys
import pkg_resources
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 30)

import plotly.plotly as py
import plotly.graph_objs as go
# plotly.offline.init_notebook_mode(connected=True)

print ('Python:', sys.version)
print ('NumPy:', np.__version__)
print ('Pandas:', pd.__version__)
print ('Plotly:', pkg_resources.get_distribution("plotly").version)

Python: 3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609]
NumPy: 1.13.0
Pandas: 0.20.2
Plotly: 2.0.11


Let's check how much data do we have to download

In [2]:
import urllib.request
site1, site2 = urllib.request.urlopen('http://stat-computing.org/dataexpo/2009/2008.csv.bz2'),\
               urllib.request.urlopen('http://stat-computing.org/dataexpo/2009/2007.csv.bz2')
print('Total size of compressed datasets to be downloaded: {:.2f} MB'.format( int(site1.length + site2.length) / float(1 << 20)))

Total size of compressed datasets to be downloaded: 224.12 MB


Grab datasets

In [4]:
df_2008 = pd.read_csv('http://stat-computing.org/dataexpo/2009/2008.csv.bz2', compression='bz2', parse_dates=[[0,1,2]])
df_2007 = pd.read_csv('http://stat-computing.org/dataexpo/2009/2007.csv.bz2', compression='bz2', parse_dates=[[0,1,2]])

Let's check column integrity before concatenating datasets

In [5]:
(df_2008.columns == df_2007.columns).all()

True

In [6]:
df = pd.concat([df_2008, df_2007])
del df_2008, df_2007 # just in case, let's free some memory

Check if data from both years was imported

In [7]:
pd.DatetimeIndex(df['Year_Month_DayofMonth']).year.value_counts()

2007    7453215
2008    7009728
Name: Year_Month_DayofMonth, dtype: int64

### we successtuflly created dataset with 2007 and 2008 data

Let's explore our dataset

In [8]:
df.shape

(14462943, 27)

In [9]:
df.head()

,Year_Month_DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008-01-03,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [10]:
df.columns

Index(['Year_Month_DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

## 2. Scharakteryzuj wybranego przewoźnika (UniqueCarrier) na przestrzeni tych dwóch lat na podstawie dostępnych danych.

Let's do a qucik analysis of <b>cancelled flights</b> for carrier with most number of scheduled flights

In [11]:
df.UniqueCarrier.value_counts().nlargest(5)

WN    2370625
AA    1238742
OO    1165041
MQ    1031187
UA     939517
Name: UniqueCarrier, dtype: int64

Who is "WN"?

In [12]:
df_carriers = pd.read_csv('http://stat-computing.org/dataexpo/2009/carriers.csv')
# df_carriers[df_carriers.Code.isin(df.UniqueCarrier.unique())] # pd.DafaFrame with codes and names of all airlines in our data
df_carriers[df_carriers.Code == 'WN']

,Code,Description
1388,WN,Southwest Airlines Co.


Let's plot percentage of <b>cancelled flights/total count</b> for each carrier

In [13]:
df_my_airline = df[df.UniqueCarrier == 'WN']
# s_cancelled will hold a pd.Series with indexes:UniqueCarrier(two-character string code) and values:ratio between cancelled flights over total flights(float, rounded)  
s_cancelled = (100 * df.groupby('UniqueCarrier')['CancellationCode'].count() / df.UniqueCarrier.value_counts()).round(2) # type(s_cancelled) = pd.Series

df_carriers_cancelledFlights = df_carriers.merge(s_cancelled.to_frame(), 
                                                 left_on='Code', 
                                                 right_index=True).drop('Code', axis=1)\
                                                                  .rename_axis(mapper={0: 'CancelledFlights'}, axis='columns')\
                                                                  .sort_values(by='CancelledFlights')

df_carriers_cancelledFlights

,Description,CancelledFlights
517,Frontier Airlines Inc.,0.36
619,Hawaiian Airlines Inc.,0.68
206,Aloha Airlines Inc.,0.79
537,AirTran Airways Corporation,0.92
1388,Southwest Airlines Co.,0.94
377,Continental Air Lines Inc.,1.07
920,Northwest Airlines Inc.,1.41
441,Delta Air Lines Inc.,1.44
221,Alaska Airlines Inc.,1.51
1308,US Airways Inc. (Merged with America West 9/05...,1.65


In [83]:
# link to online copy: https://plot.ly/~lekkiduchem/4/percentage-of-cancelled-flights-between-competitors/

data = [go.Bar(x=df_carriers_cancelledFlights.Description.values, 
               y=df_carriers_cancelledFlights.CancelledFlights.values, 
               marker = dict(color=['rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(222,45,38,0.8)',
                                    'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)',
                                    'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)',
                                    'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(204,204,204,1)' ]
                            ))]

layout = go.Layout(title="Percentage of cancelled flights between competitors",
                   autosize=True,
                   direction=60,
                   margin=go.Margin(l=70,
                                    r=70,
                                    b=120)
                   )
                 
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cancelled flights by company')

Let's check value distribution between 4 groups of different cancellation codes for cancelled flights
<ol type="A">
    <li>Carrier</li>
    <li>Weather</li>
    <li>National Air System</li>
    <li>Security</li>
</ol>

In [17]:
df_my_airline_cancelled = df_my_airline.CancellationCode.value_counts().to_frame()\
                                                                       .rename_axis(mapper={'A': 'Carrier',
                                                                                            'B': 'Weather',
                                                                                            'C': 'National Air System',
                                                                                            'D': 'Security'}
                                                                                   )
df_my_airline_cancelled

,CancellationCode
Carrier,12314
Weather,8954
National Air System,1096
Security,18


In [86]:
# link to online copy: https://plot.ly/~lekkiduchem/2/southwest-airlines-cancelled-flights-by-reason/

values = df_my_airline_cancelled.values
labels = df_my_airline_cancelled.index
colors = ['rgba(222,45,38,0.8)', 'rgba(204,204,204,1)', 'rgba(194,194,194,1)', 'rgba(184,184,184,1)']

trace = [go.Pie(values=values,
                labels=labels, 
                marker=dict(colors=colors))]

layout = go.Layout(title="Southwest Airlines cancelled flights by reason")

fig = go.Figure(data=trace, layout=layout)
py.iplot(fig, filename='WN-cancelled-flights')

## 3. Wiążąc kilka zmiennych ze sobą, wykaż ciekawą (we własnej ocenie) zależność dla jednego carriera lub dla całej grupy

### 3.A TOP 10 Most connected cieties

In [19]:
df_airports = pd.read_csv('http://stat-computing.org/dataexpo/2009/airports.csv')
df_airports[df_airports.iata == 'ATL']

,iata,airport,city,state,country,lat,long
880,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944


In [33]:
# Origin and Dest
df_airports_connections = pd.concat([df.Origin, df.Dest], axis=1).rename_axis(mapper={'Origin':'Origin_iata', 'Dest':'Dest_iata'}, axis='columns')
connections = df_airports_connections.groupby(['Origin_iata', 'Dest_iata']).groups
connections_reversed = df_airports_connections.groupby(['Dest_iata', 'Origin_iata']).groups
d = {}

for origin, dest in connections:
    if (origin, dest) in connections_reversed: 
        d[origin, dest] = np.append(connections[origin, dest], connections_reversed[origin, dest]).tolist()
        
pd.Series(d)

ABE  ATL    [152435, 153094, 153550, 154101, 154755, 15531...
     BWI                                    [4530693, 155074]
     CLE    [102898, 103677, 103969, 104435, 104621, 10471...
     CLT    [256204, 257484, 258820, 260158, 261504, 26262...
     CVG    [151383, 152063, 152721, 153288, 154386, 15499...
     DTW    [457674, 457675, 457676, 457677, 457678, 45767...
     ORD    [129113, 129114, 129800, 129801, 129802, 13054...
ABI  DFW    [383352, 383353, 383354, 383355, 383356, 38335...
ABQ  AMA    [2367, 5805, 8614, 11843, 15279, 18715, 22151,...
     ATL    [302992, 303203, 303947, 304232, 304451, 30521...
     AUS    [96836, 96837, 96838, 96839, 96840, 96841, 968...
     BWI    [2368, 5806, 8615, 11844, 15280, 18716, 22152,...
     CLE    [3109245, 3111332, 3120365, 3716592, 3717543, ...
     CVG    [303902, 305167, 306473, 308764, 309938, 31116...
     DAL    [2369, 2370, 2371, 2372, 2373, 2374, 2375, 237...
     DEN    [2378, 2379, 2380, 5814, 5815, 5816, 8625, 862...
     DFW

In [22]:
s_connections = pd.Series(d).count(level=0).nlargest(10)
s_connections

ATL    187
ORD    150
DFW    135
MSP    130
DEN    122
CVG    119
DTW    118
IAH    113
SLC    113
LAS     92
dtype: int64

In [23]:
df_airports.loc[df_airports.iata.isin(s_connections.index), ['city', 'state', 'airport']].reset_index(drop=True)

,city,state,airport
0,Atlanta,GA,William B Hartsfield-Atlanta Intl
1,Covington,KY,Cincinnati Northern Kentucky Intl
2,Denver,CO,Denver Intl
3,Dallas-Fort Worth,TX,Dallas-Fort Worth International
4,Detroit,MI,Detroit Metropolitan-Wayne County
5,Houston,TX,George Bush Intercontinental
6,Las Vegas,NV,McCarran International
7,Minneapolis,MN,Minneapolis-St Paul Intl
8,Chicago,IL,Chicago O'Hare International
9,Salt Lake City,UT,Salt Lake City Intl


In [80]:
# link to online copy: https://plot.ly/~lekkiduchem/14/top-10-most-connected-airports-in-usa/

airports = [ dict(type = 'scattergeo',
                  locationmode = 'USA-states',
                  lat = df_airports.loc[df_airports.iata.isin(s_connections.index), ['lat']].values,
                  lon = df_airports.loc[df_airports.iata.isin(s_connections.index), ['long']].values,
                  hoverinfo = 'text',
                  text = df_airports.loc[df_airports.iata.isin(s_connections.index), ['airport']].values,
                  mode = 'markers',
                  marker = dict(size=10, 
                                color='rgb(255, 0, 0)',
                                line = dict(width=1, color='rgba(68, 68, 68, 0)')
                               )
                 )]

layout = dict(
        title = 'TOP 10 Most connected airports in USA',
        showlegend = False, 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=airports, layout=layout )
py.iplot( fig, filename='most-connected-usa-airports' )

### 3.B Map of air connections
*I always wanted to make one*

In [34]:
df_airports_connections.head()

,Origin_iata,Dest_iata
0,IAD,TPA
1,IAD,TPA
2,IND,BWI
3,IND,BWI
4,IND,BWI


In [35]:
df_airports_geoloc = df_airports.loc[:,['iata', 'lat', 'long']]
df_airports_geoloc.head()

,iata,lat,long
0,00M,31.953765,-89.234505
1,00R,30.685861,-95.017928
2,00V,38.945749,-104.569893
3,01G,42.741347,-78.052081
4,01J,30.688012,-81.905944


In [42]:
df_airports_connections = pd.concat([df.Origin, df.Dest], axis=1).rename_axis(mapper={'Origin':'Origin_iata', 'Dest':'Dest_iata'}, axis='columns')

df_airports_connections = df_airports_connections.merge(df_airports_geoloc, 
                                                        left_on='Origin_iata', 
                                                        right_on='iata', 
                                                        how='left').drop(['Origin_iata', 'iata'], axis=1)\
                                                                            .rename_axis(axis='columns',
                                                                                         mapper={'lat': 'start_lat',
                                                                                                 'long': 'start_lon'})

df_airports_connections = df_airports_connections.merge(df_airports_geoloc, 
                                                        left_on='Dest_iata', 
                                                        right_on='iata', 
                                                        how='left').drop(['Dest_iata', 'iata'], axis=1)\
                                                                            .rename_axis(axis='columns',
                                                                                         mapper={'lat': 'end_lat',
                                                                                                 'long': 'end_lon'}) 

df_airports_connections.head()

,start_lat,start_lon,end_lat,end_lon
0,38.944532,-77.455810,27.975472,-82.533250
1,38.944532,-77.455810,27.975472,-82.533250
2,39.717329,-86.294384,39.175402,-76.668198
3,39.717329,-86.294384,39.175402,-76.668198
4,39.717329,-86.294384,39.175402,-76.668198


In [77]:
# link to online copy: https://plot.ly/~lekkiduchem/12/map-of-random-air-connections/ 

airports = [ dict(type = 'scattergeo',
                  locationmode = 'USA-states',
                  lat = df_airports['lat'],
                  lon = df_airports['long'],
                  hoverinfo = 'text',
                  text = df_airports['airport'],
                  mode = 'markers',
                  marker = dict(size=1, 
                                color='rgb(255, 0, 0)',
                                line = dict(width=3, color='rgba(68, 68, 68, 0)')
                               )
                 )]

df_flights = df_airports_connections.sample(500)
flight_paths = []
for i in range( len(df_fligts) ):
    flight_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [ df_flights.loc[df_flights.index[i], 'start_lon'], df_flights.loc[df_flights.index[i], 'end_lon'] ],
            lat = [ df_flights.loc[df_flights.index[i], 'start_lat'], df_flights.loc[df_flights.index[i], 'end_lat'] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            )
#             opacity = float(df_airports_connections['cnt'][i])/float(df_airports_connections['cnt'].max()),
        )
    )
    
layout = dict(
        title = 'Map of random air connections',
        showlegend = False, 
        geo = dict(
            scope='north america',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict( data=flight_paths + airports, layout=layout )
py.iplot( fig, filename='d3-flight-paths' )

## 4. What would be really fun to do
<p>How awesome would it be, if we could create an algorithm which would predict likelyhood of our flight being cancelled/delayed. We could use dozens of features like: weather, airline performance metrics, day of the week, holiday, number of flights which are being handled by the airport on that day. </p>  

<p>We would have to gather historical data for all as many past flights as possible, clean it up and feed to some neural network, which would train on the data. We would use our features to feed import neurons in input layer, in odered to compute value of single neuron in output layer, to give us the likelyhood</p>